In [1]:
import pyspark
import math

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [49]:
texto1 = sc.textFile('./texto1.txt')

In [50]:
texto1.collect()

['hola que tal', 'que tal todos', 'hola a todes', 'hola a todxs']

In [6]:
texto2 = texto1.flatMap(lambda x: x.split())
texto2.collect()

['hola', 'que', 'tal', 'que', 'tal', 'todos', 'hola', 'a', 'todos']

In [7]:
texto3 = texto2.map(lambda x: (x,1))
texto3.collect()

[('hola', 1),
 ('que', 1),
 ('tal', 1),
 ('que', 1),
 ('tal', 1),
 ('todos', 1),
 ('hola', 1),
 ('a', 1),
 ('todos', 1)]

In [10]:
texto4 = texto3.reduceByKey(lambda x,y: x + y)
texto4.collect()

[('hola', 2), ('que', 2), ('todos', 2), ('tal', 2), ('a', 1)]

In [23]:
def ngramas(linea, n):
    linea = linea.lower()
    ngramas = []
    for i in range(0, len(linea) - n - 1):
        ngramas.append(linea[i:i+n])
    return ngramas

In [24]:
ngramas('hola que tal', 3)

['hol', 'ola', 'la ', 'a q', ' qu', 'que', 'ue ', 'e t']

In [26]:
texto5 = texto1.flatMap(lambda x: ngramas(x,3))
texto5.collect()

['hol',
 'ola',
 'la ',
 'a q',
 ' qu',
 'que',
 'ue ',
 'e t',
 'que',
 'ue ',
 'e t',
 ' ta',
 'tal',
 'al ',
 'l t',
 ' to',
 'tod',
 'hol',
 'ola',
 'la ',
 'a a',
 ' a ',
 'a t',
 ' to',
 'tod']

In [27]:
texto6 = texto5.map(lambda x: (x, 1))\
               .reduceByKey(lambda x,y: x + y)
texto6.collect()

[('hol', 2),
 ('la ', 2),
 ('que', 2),
 ('ue ', 2),
 ('e t', 2),
 (' ta', 1),
 ('l t', 1),
 ('a a', 1),
 (' a ', 1),
 ('a t', 1),
 ('ola', 2),
 ('a q', 1),
 (' qu', 1),
 ('tal', 1),
 ('al ', 1),
 (' to', 2),
 ('tod', 2)]

In [29]:
texto7 = texto6.sortBy(ascending=False, keyfunc = lambda x: x[1])
texto7.collect()

[('hol', 2),
 ('la ', 2),
 ('que', 2),
 ('ue ', 2),
 ('e t', 2),
 ('ola', 2),
 (' to', 2),
 ('tod', 2),
 (' ta', 1),
 ('l t', 1),
 ('a a', 1),
 (' a ', 1),
 ('a t', 1),
 ('a q', 1),
 (' qu', 1),
 ('tal', 1),
 ('al ', 1)]

In [17]:
total = texto7.map(lambda x: x[1])\
              .reduce(lambda x,y: x + y)
total

25

In [22]:
result = texto7.map(lambda x: (x[0], x[1]/total))
result.collect()

[('hol', 0.08),
 ('la ', 0.08),
 ('que', 0.08),
 ('ue ', 0.08),
 ('e t', 0.08),
 ('ola', 0.08),
 (' to', 0.08),
 ('tod', 0.08),
 (' ta', 0.04),
 ('l t', 0.04),
 ('a a', 0.04),
 (' a ', 0.04),
 ('a t', 0.04),
 ('a q', 0.04),
 (' qu', 0.04),
 ('tal', 0.04),
 ('al ', 0.04)]

In [42]:
def ngramas_contexto(linea, n=3, contexto=1):
    linea = linea.lower()
    ngramas = []
    for i in range(0, len(linea) - n - 1):
        ngramas.append((linea[i:i+n],linea[i+n:i+n+contexto]))
    return ngramas

In [37]:
ngramas_contexto('hola que tal', 3, 1)

[('hol', 'a'),
 ('ola', ' '),
 ('la ', 'q'),
 ('a q', 'u'),
 (' qu', 'e'),
 ('que', ' '),
 ('ue ', 't'),
 ('e t', 'a')]

In [51]:
texto1.collect()

['hola que tal', 'que tal todos', 'hola a todes', 'hola a todxs']

In [52]:
contexto = texto1.flatMap(lambda x: ngramas_contexto(x))
contexto.collect()

[('hol', 'a'),
 ('ola', ' '),
 ('la ', 'q'),
 ('a q', 'u'),
 (' qu', 'e'),
 ('que', ' '),
 ('ue ', 't'),
 ('e t', 'a'),
 ('que', ' '),
 ('ue ', 't'),
 ('e t', 'a'),
 (' ta', 'l'),
 ('tal', ' '),
 ('al ', 't'),
 ('l t', 'o'),
 (' to', 'd'),
 ('tod', 'o'),
 ('hol', 'a'),
 ('ola', ' '),
 ('la ', 'a'),
 ('a a', ' '),
 (' a ', 't'),
 ('a t', 'o'),
 (' to', 'd'),
 ('tod', 'e'),
 ('hol', 'a'),
 ('ola', ' '),
 ('la ', 'a'),
 ('a a', ' '),
 (' a ', 't'),
 ('a t', 'o'),
 (' to', 'd'),
 ('tod', 'x')]

In [53]:
contexto = contexto.map(lambda x: (x, 1))\
                   .reduceByKey(lambda x,y: x + y)
contexto.collect()

[(('ola', ' '), 3),
 (('a q', 'u'), 1),
 ((' qu', 'e'), 1),
 ((' ta', 'l'), 1),
 (('tal', ' '), 1),
 (('al ', 't'), 1),
 (('tod', 'o'), 1),
 (('tod', 'e'), 1),
 (('tod', 'x'), 1),
 (('hol', 'a'), 3),
 (('la ', 'q'), 1),
 (('que', ' '), 2),
 (('ue ', 't'), 2),
 (('e t', 'a'), 2),
 (('l t', 'o'), 1),
 ((' to', 'd'), 3),
 (('la ', 'a'), 2),
 (('a a', ' '), 2),
 ((' a ', 't'), 2),
 (('a t', 'o'), 2)]

In [72]:
c = contexto.map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().mapValues(list)
c.collect()

[(' ta', [('l', 1)]),
 ('hol', [('a', 3)]),
 ('la ', [('q', 1), ('a', 2)]),
 ('que', [(' ', 2)]),
 ('ue ', [('t', 2)]),
 ('e t', [('a', 2)]),
 ('l t', [('o', 1)]),
 ('a a', [(' ', 2)]),
 (' a ', [('t', 2)]),
 ('a t', [('o', 2)]),
 ('ola', [(' ', 3)]),
 ('a q', [('u', 1)]),
 (' qu', [('e', 1)]),
 ('tal', [(' ', 1)]),
 ('al ', [('t', 1)]),
 ('tod', [('o', 1), ('e', 1), ('x', 1)]),
 (' to', [('d', 3)])]

In [69]:
def suma(lista):
    res = 0
    for i in range(0,len(lista)):
        res += lista[i][1]
    return res

In [71]:
suma([('o', 5), ('e', 1), ('x', 1)])

7

In [74]:
d = c.map(lambda x: (x[0], x[1], suma(x[1])))
d.collect()

[(' ta', [('l', 1)], 1),
 ('hol', [('a', 3)], 3),
 ('la ', [('q', 1), ('a', 2)], 3),
 ('que', [(' ', 2)], 2),
 ('ue ', [('t', 2)], 2),
 ('e t', [('a', 2)], 2),
 ('l t', [('o', 1)], 1),
 ('a a', [(' ', 2)], 2),
 (' a ', [('t', 2)], 2),
 ('a t', [('o', 2)], 2),
 ('ola', [(' ', 3)], 3),
 ('a q', [('u', 1)], 1),
 (' qu', [('e', 1)], 1),
 ('tal', [(' ', 1)], 1),
 ('al ', [('t', 1)], 1),
 ('tod', [('o', 1), ('e', 1), ('x', 1)], 3),
 (' to', [('d', 3)], 3)]

In [78]:
def proba(lista, total):
    l = []
    for i in range(0, len(lista)):
        l.append((lista[i][0],lista[i][1] / total))
    return l

In [79]:
proba([('o', 5), ('e', 1), ('x', 1)], 7)

[('o', 0.7142857142857143),
 ('e', 0.14285714285714285),
 ('x', 0.14285714285714285)]

In [83]:
d.map(lambda x: (x[0], proba(x[1],x[2]))).collect()

[(' ta', [('l', 1.0)]),
 ('hol', [('a', 1.0)]),
 ('la ', [('q', 0.3333333333333333), ('a', 0.6666666666666666)]),
 ('que', [(' ', 1.0)]),
 ('ue ', [('t', 1.0)]),
 ('e t', [('a', 1.0)]),
 ('l t', [('o', 1.0)]),
 ('a a', [(' ', 1.0)]),
 (' a ', [('t', 1.0)]),
 ('a t', [('o', 1.0)]),
 ('ola', [(' ', 1.0)]),
 ('a q', [('u', 1.0)]),
 (' qu', [('e', 1.0)]),
 ('tal', [(' ', 1.0)]),
 ('al ', [('t', 1.0)]),
 ('tod',
  [('o', 0.3333333333333333),
   ('e', 0.3333333333333333),
   ('x', 0.3333333333333333)]),
 (' to', [('d', 1.0)])]